<b>Pandas之数据分组和聚合</b>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'key1':['a','a','b','b','a'],'key2':['one','two','one','two','one'],'data1':np.random.randn(5),'data2':np.random.randn(5)})
df

,data1,data2,key1,key2
0,0.614349,0.293624,a,one
1,2.073042,-0.255644,a,two
2,-1.220986,-0.700115,b,one
3,-0.549256,0.468301,b,two
4,-0.258352,0.485996,a,one


<b>数据分组（groupby）</b>

.groupby(by=None, axis=0, level=None, as_index=True, sort=True, group_keys=True, squeeze=False)方法作用于一条轴向上，并接受一个分组键（by）参数来给调用者分组。分组键可以是Series 或列表，要求其长度与待分组的轴一致；也可以是映射函数、字典甚至数组的某条列名（字符串），但这些参数类型都只是快捷方式，其最终仍要用于生成一组用于拆分对象的值。

In [3]:
grouped = df.groupby(df['key1'])
grouped

这里使用 df['key1'] 做了分组键，即按 a 和 b 进行分组。但实际分组键并不需要与数组对象之间存在联系，只要长度相同即可，使用数组的列只是图方便。上例中如果使用 [1,1,2,2,3] 这样的列表做分组键的话，结果与 df['key1'] 是相同的。  
roupby 方法的调用本身并不涉及运算，因此速度很快。而在操作这个 grouped 对象的时候，还是将其看成一个保存了实际数据的对象比较方便。比如我们可以直接对其应用很多方法，或索引切片：

In [5]:
grouped.mean()

,data1,data2
key1,,
a,0.809679,0.174659
b,-0.885121,-0.115907


上例中没有显示 key2 列，是因为其值不是数字类型，被 mean() 方法自动忽视了。当想要只看某一（些）列的时候，可以通过索引来实现，在 groupby 方法调用前后均可

In [6]:
df.groupby(df['key1'])['data1'].mean()

key1
a    0.809679
b   -0.885121
Name: data1, dtype: float64

如果分组键使用的是多个数组，就会得到一个层次化索引的结果：

In [7]:
df.groupby([df['key1'],df['key2']]).mean()

data1     data2
key1 key2                    
a    one   0.177998  0.389810
     two   2.073042 -0.255644
b    one  -1.220986 -0.700115
     two  -0.549256  0.468301

最后，可以使用 GroupBy 对象（不论是 DataFrameGroupBy 还是 SeriesGroupBy）的 .size() 方法查看分组大小：

In [8]:
grouped.size()

key1
a    3
b    2
dtype: int64

GroupBy 对象是可以通过 for 循环迭代的，可以产生一组二元组，分别为分组名和组内数据。下面是一个多重分组键的情况：

In [9]:
for i,j in df.groupby([df['key1'],df['key2']]):
    print(i)
    print('-----------')
    print(j)

('a', 'one')
-----------
      data1     data2 key1 key2
0  0.614349  0.293624    a  one
4 -0.258352  0.485996    a  one
('a', 'two')
-----------
      data1     data2 key1 key2
1  2.073042 -0.255644    a  two
('b', 'one')
-----------
      data1     data2 key1 key2
2 -1.220986 -0.700115    b  one
('b', 'two')
-----------
      data1     data2 key1 key2
3 -0.549256  0.468301    b  two


<b>使用字典或Series作分组键</b>

这两种参数需要提供一种从行（列）名到组名的映射关系。

In [10]:
df.groupby({0:'a',1:'a',2:'b',3:'b',4:'a'}).mean()

,data1,data2
a,0.809679,0.174659
b,-0.885121,-0.115907


<b>通过函数进行分组</b>

函数的作用有些类似于字典，或者说这些奇怪的分组键都类似于字典——利用某种映射关系将待分组的轴转化为一个等长的由分组名组成的序列。

如果说行列名是作为索引传递给字典以获取组名的话，那么在函数分组键中，行列名就会作为参数传递给函数。这便是你需要提供的函数类型：

In [11]:
df.groupby(lambda x:'even' if x%2==0 else 'odd').mean()

,data1,data2
even,-0.288330,0.026502
odd,0.761893,0.106329


<b>根据索引级别分组</b>

当根据高级别索引来分组的时候，参数就不再是 by=None 了，而要换成 level=None，值可以是索引级别的编号或名称：

In [12]:
index = pd.MultiIndex.from_arrays([['even','odd','even','odd','even'],[0,1,2,3,4]],names=['a','b'])
df.index = index
df

,,data1,data2,key1,key2
a,b,,,,
even,0,0.614349,0.293624,a,one
odd,1,2.073042,-0.255644,a,two
even,2,-1.220986,-0.700115,b,one
odd,3,-0.549256,0.468301,b,two
even,4,-0.258352,0.485996,a,one


In [13]:
df.groupby(level='a').mean()

,data1,data2
a,,
even,-0.288330,0.026502
odd,0.761893,0.106329


In [14]:
 df.groupby(level=0).mean()

,data1,data2
a,,
even,-0.288330,0.026502
odd,0.761893,0.106329


<b>数据聚合（Aggregation）</b>

数据聚合，指的是任何能够从数组产生标量值的数据转换过程。你也可以简单地将其理解为统计计算，如 mean(), sum(), max() 等。

数据聚合本身与分组并没有直接关系，在任何一列（行）或全部列（行）上都可以进行。不过当这种运算被应用在分组数据上的时候，结果可能会变得更有意义。

对于 GroupBy 对象可以应用的聚合运算包括：

　已经内置的方法，如 sum()， mean() 等  
　Series 的方法，如 quantile() 等  
　自定义的聚合函数，通过传入 GroupBy.aggregate() 或 GroupBy.agg() 来实现  

其中自定义函数的参数应当为一个数组类型，即 GroupBy 对象迭代出的元组的第二个元素。如

http://my.oschina.net/lionets/blog/280332